In [ ]:
import requests
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.inspection import PartialDependenceDisplay
from sklearn import metrics

In [ ]:
import requests
from pathlib import Path
def download_file(url):

    data = requests.get(url)

    with open(Path(url).name, 'wb') as f:
        f.write(data.content)
download_file('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv')
COVID_DATA = pd.read_csv('owid-covid-data.csv')

In [ ]:
COVID_DATA.describe()

In [ ]:
COVID_DATA.describe(include="O")
COVID_DATA['continent'].unique()

In [ ]:
COVID_DATA.shape
COVID_DATA.iloc[-1]

In [ ]:
plt.style.use('ggplot')
l=list()
for i in range(COVID_DATA.shape[0]):
    if type(COVID_DATA.continent[i])==float and COVID_DATA.location[i] in ["North America","South America","Asia"]:
        l.append(i)
subdata=COVID_DATA[["continent","location","date","new_vaccinations",'new_vaccinations_smoothed_per_million',"new_people_vaccinated_smoothed_per_hundred",'total_boosters_per_hundred','new_deaths_per_million']].iloc[l]
subdata.groupby("location").plot(x="date",y="new_deaths_per_million",kind="line",figsize=(8,5),color='b')#project


The above graphs displays the rate of the number of death new COVID-19 cases per million persons in Asia, North America, and South America from 2020/01 to 2023/04, respectively. We can see the death number is periodic for Asia and North America. Around a quater of year would appear a peak in Asia and half year a peak in North America. And the growth peaks in South America occurred in 2021/07 and 2022/03. For the total number of death, North America is around 5x of Asia and South America is 15x of Asia.

In [ ]:
subdata.groupby("location").plot(x="new_vaccinations",y="new_deaths_per_million",kind="scatter",figsize=(8,5),color='r')

From the below graphs, they display the number of death number new COVID-19 cases per million persons with the new people vaccinatedin Asia, North America and South America from 2020/1 to 2023/04, respectively. We can see that the most remarkable change is in North America. With the new vaccine injection, new cases goes down sharply and become stable. However, for Asia, there is no significant changes with the increase of new vaccinations.

In [ ]:
subdata.groupby("location").plot(x="total_boosters_per_hundred",y="new_deaths_per_million",kind="scatter",figsize=(8,5),color='r')#project

The above graphs display they display the number of death number new COVID-19 cases per million persons with the total booster shot per hundred of people shot in Asia, North America and South America from 2020/1 to 2023/04, respectively. As we can see, for Asia and North America, with the increasing number of total boosters, the new death number clearly goes down. However, when it reaches a steady state, the Covid disease impacts on new death number again, which is around 35-40 total boosters. For South America, the new death numbers appear to be stable.


In [ ]:
subdata.groupby("location").plot(x="new_people_vaccinated_smoothed_per_hundred",y="new_deaths_per_million",kind="scatter",figsize=(8,5),color='r')#project

From the above graphs, they display the number of death number new COVID-19 cases per million persons with the new people vaccinated smoothed per hundred in Asia, North America and South America from 2020/1 to 2023/04, respectively. We can see that the most remarkable change is in North America. With the new vaccine injection, new cases goes down sharply and become stable.

In [ ]:
subdata2=COVID_DATA.loc[(COVID_DATA.location=="Brazil") | (COVID_DATA.location=="Japan") | (COVID_DATA.location=="United States")]
Location=np.sort(subdata2["location"].unique())
fig, axes = plt.subplots(1, len(Location), figsize=(24, 4), sharey=True)
for ax, loca in zip(axes, Location):
    ax.scatter(subdata2[subdata2['location']==loca]['stringency_index'],subdata2[subdata2['location']==loca]['new_deaths_per_million'])
    ax.set_xlabel('stringency_index')
    ax.set_ylabel('new_deaths_per_million')
    ax.set_title(loca)#project

The above three scatter plots show daily new COVID-19 death cases per million people versus the severity of the government's response between 2020/1 to 2023/04 in Brazil, Japan and United States,respectively. It can be seen that the daily new death cases per million in Japan have been relatively low, and there is no obvious dependency. For Brazil, there are many more new daily disease cases than in Japan and there is clear relationship between the government response and daily new cases per million, which is more frequent government response with more cases of death. For United States, as you can see from the plot, daily new cases stable when the government response index is around 30. And evey new cases death reach a peak, when government response, the death cases goes down for a while. In order to further observe the relationship between Japan's government response index and daily new death cases, the above scatter plots are drawn separately.


In [ ]:
plt.scatter(subdata2[subdata2['location']=="Japan"]['stringency_index'],subdata2[subdata2['location']=="Japan"]['new_deaths_per_million'])
plt.xlabel('stringency_index')
plt.ylabel('new_deaths_per_million')
plt.title("Japan")#project

Although Japan's daily new mortality cases per million have been very low, we may observe a very intriguing phenomenon: when the daily new cases per million are pretty high, the government response index is very high. We can realize that when the government acts, the epidemic frequently gets worse.

Using the aforementioned analysis, we can infer some preliminary conclusions about the dataset. Over the previous three years, South America has been the region most affected, followed by North America and then Asia (The indicator assessed is the daily new death cases for covid-19 per million)
In South America and Asia, increased vaccination and booster coverage have helped suppress outbreaks, but in North America, they haven't made much of a difference.
In some ways, the country's response to the pandemic is reflected in the relationship between the government's response index and the number of daily new cases per million.

# Model1: Linear Regression

The linear regression model we proposed is :
$$E(Y)=\beta_0+\beta_1 X_1+ \beta_2 X_2+\beta_3 X_2^2$$
The explanatory variables $X_i$'s ($i=1,2,3$) are stringency_index, new_people_vaccinated_smoothed_per_hundred in Japan and the response variable $Y$ is the new_deaths_per_million in Japan.

The methodolody here to estimated the coefficients is OLS.

In [ ]:
japan = COVID_DATA.loc[COVID_DATA.location == "Japan"]
japan_data = japan[["new_deaths_per_million", "stringency_index", "new_people_vaccinated_smoothed_per_hundred","total_boosters_per_hundred"]]
japan_data.dropna(inplace=True)

X = japan_data[["stringency_index","total_boosters_per_hundred", "new_people_vaccinated_smoothed_per_hundred"]]
X["squared_vaccinated"] = X["new_people_vaccinated_smoothed_per_hundred"] ** 2
X_values = X.values

Y_values = japan_data[["new_deaths_per_million"]].values
X_train, X_test, Y_train, Y_test = train_test_split(X_values, Y_values, test_size=0.1, random_state=132)

model = LinearRegression().fit(X_train, Y_train)
print('coefficient of determination:', model.score(X_train,Y_train))
print("model coefficients are:",model.coef_)
print("model intercept is:",model.intercept_)
y_pred = model.predict(X_test)
MSE=metrics.mean_squared_error(Y_test, y_pred)
RMSE = MSE**0.5
MAE=metrics.mean_absolute_error(Y_test, y_pred)
print("RMSE:",RMSE)
print("MSE:",MSE)
print("MAE:",MAE)

Therefore, the estimated regression model is $$E(Y)=-0.0404X_1+0.1628X_2-0.1464X_2^2$$

The RMSE, MSE and MAE on the test dataset are 0.6192, 0.3834 and 0.5079 respectively.

In [ ]:
# check residual normality
import statsmodels.api as sm
import scipy.stats as stats
residuals=Y_train - model.predict(X_train)
Shapiro_test =stats.shapiro(residuals)
print("The Shapiro test for the normality is:",Shapiro_test[0])
print("The p-value of the test is:",(Shapiro_test[1]))
fitted=model.predict(X_train)
plt.figure()
plt.scatter(fitted,residuals)
plt.xlabel("Fitted Values")
plt.ylabel("Residuals")
plt.show()

In [ ]:
sns.kdeplot(data=residuals ,fill= True,bw_adjust=1.5 )

The result of Shapiro test for the normality as follows:0.9648 with the p-value of 0.000000 ( significantly smaller than any significance level). Our interpretation for the result is the residuals does not follow normal distribution

Now we are checking the assumption of homogeneity of variance. According to the residual vs fitted value plot, we observed that the dots are randomly scattered, which satisfies the criterion of homogeneous variance. Hence, the assumption is not violated.

# Multiple Linear Regression

We proposed another model: the multiple linear regression model:

$$E(Y)=\beta_0+\beta_1 X_1+ \beta_2 X_2+\beta_3 X_3 + \beta_4 X_4$$

$X_1$ is the number of people older than 70 in Japan, $X_2$ is the the gdp per capita in Japan, $X_3$ is the total cases percent in Japan, $X_4$ is the population in Japan, $Y$ is the total deaths in Japan.


In [ ]:

import warnings
warnings.filterwarnings("ignore")

covid = japan

covid=covid.loc[pd.isna(covid["continent"])==False]
covid=covid.loc[covid.total_cases.isna()==False]
covid_reg=covid[["total_cases","total_deaths","median_age","aged_70_older","population","stringency_index"]]
covid_reg.dropna(inplace=True)
covid_reg['total_cases_percent']=covid_reg['total_cases']*100/covid_reg['population']
covid_reg['total_deaths_percent']=covid_reg['total_deaths']*100/covid_reg['population']
covid_reg.corr()

In [ ]:

import statsmodels.api as sm
x=covid_reg[["median_age", "aged_70_older","stringency_index", "total_deaths_percent"]]
x=sm.add_constant(x)
y=covid_reg["total_cases_percent"]
model=sm.OLS(y,x).fit()
print(model.summary())
mae=np.sum(np.abs(y-model.fittedvalues))/1053
print("RMSE:",(model.mse_resid*1048/1053)**0.5)
print("MSE:",model.mse_resid*1048/1053)
print("MAE:",mae)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
wellness = covid_reg
wellness.dropna(inplace=True)
wellness=wellness.sample(wellness.shape[0]//2, ignore_index=True)
wellness = wellness.drop(columns=['population','total_cases','total_deaths'])
wellness = wellness.astype('float32')
wellness.tail(5)
xw = wellness[["median_age","aged_70_older","stringency_index","total_deaths_percent"]]
yw = wellness["total_cases_percent"]
xw = sm.add_constant(xw)
xw_train, xw_test, yw_train, yw_test = train_test_split(xw, yw, test_size=0.2,random_state=42)

In [ ]:
model_new = sm.OLS(yw_train, xw_train).fit()

yw_pred = model_new.predict(xw_test)
mse_w = np.mean(abs(yw_pred - yw_test))
mse_w

model_new.summary()

In [ ]:
residuals_w=yw_train - model_new.predict(xw_train)
sns.kdeplot(data=residuals_w ,fill= True,bw_adjust=1.5 )

In [ ]:
fitted_w=model_new.predict(xw_train)
plt.figure()
plt.scatter(fitted_w,residuals_w)
plt.xlabel("Fitted Values")
plt.ylabel("Residuals")
plt.show()

# TimeSeries

In [ ]:
from statsmodels.tsa.arima.model import ARIMA


In [ ]:
death_data = COVID_DATA.loc[COVID_DATA.location == "Japan"]
death_data.index = death_data.date
death_data = death_data.new_deaths_per_million
death_data.index = pd.to_datetime(death_data.index)

In [ ]:
death_data.plot()

In [ ]:
p = 1
d = 1
q = 0

model = ARIMA(death_data, order = (p,d,q))
model_fit = model.fit()
model_fit.summary()


In conclusion, the ARIMA(1, 1, 0) model has a low AIC and BIC, which indicates our model is well-fitted. The residual diagnostics, however, point to problems with heteroscedasticity, heavy tails, and potential normality.

In [ ]:
pred_step = 100
pred = model_fit.get_forecast(steps = pred_step)

# Confidence intervals for the forecast
confidence_intervals = pred.conf_int()

In [ ]:
confidence_intervals['lower new_deaths_per_million'] = 0

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(pred.predicted_mean.index, pred.predicted_mean, label='Prediction', color='r')
death_data.plot(color = 'blue')
plt.fill_between(confidence_intervals.index, confidence_intervals.iloc[:, 0], confidence_intervals.iloc[:, 1], color='pink')
plt.xlabel('Date')
plt.ylabel('Death per million')
plt.title('Covid Death Counts in Japan')
plt.legend()
plt.show()